In [1]:
import geopandas as gpd
import shapely as sh
import os
import pyrosm
import pandas as pd

In [2]:
os.chdir('/Users/benpage/Documents/GitHub/r5food/data/')

In [3]:
def get_extract(data_dir, location):
    osm_data = pyrosm.get_data(location, directory=data_dir)
    print("Data was downloaded to:", osm_data)
    return osm_data

In [4]:
def get_supermarkets(extract_file, boundry):
    
    boundry_geom = boundry['geometry'].values[0]
    print('Loading extract')
    
    tor_osm = pyrosm.OSM(extract_file, boundry_geom)
    print('Getting supermarkets')
    supermarkets = tor_osm.get_data_by_custom_criteria(custom_filter={'shop':['supermarket']}, 
                                               extra_attributes=['brand'], 
                                               keep_relations=False)
    supermarkets['geometry'] = supermarkets['geometry'].to_crs('EPSG:26917').centroid
    return supermarkets

In [7]:
if __name__ == '__main__':
    os.chdir('/Users/benpage/Documents/GitHub/r5food/data/')
    extract_name = 'Toronto.osm.pbf'
    location = 'Toronto'
    data_dir = '/Users/benpage/Documents/GitHub/r5food/data/'

    # tor_boundry = gpd.read_file('tor_boundry.geojson')
    # supermarkets = get_supermarkets(extract_name, tor_boundry)
    # market_filter = supermarkets[['id', 'tags', 'addr:city', 'addr:housenumber', 'addr:postcode', 
    #      'addr:place', 'addr:street', 'operator', 'phone', 'ref', 'website',
    #      'brand', 'geometry', 'osm_type', 'alcohol']].sort_values(by='brand')
    # market_filter.to_file('supermarkets.geoJSON')

Loading extract
Getting supermarkets


In [10]:
market_filter.to_file('supermarkets.geoJSON')